# 00: Collect data 

This assums that you have downloaded projects data file from the [RePORTER](https://reporter.nih.gov/exporter/projects) and the European Research Council [ERC](https://erc.europa.eu/homepage) website. Make sure that there is a data folder in the project root folder in order for this notebook to work. 

Check list:
- Make sure you have a `data` folder in root project folder
- Make sure that you have downloaded the RePORTER dataset and it is stored within the project root folder

This notebook will merge all dataset and will output a compressed one. 

In [1]:
import pathlib 
import pandas as pd

## Parameters
Here are the parameters used to run this notebook:
- `PREFIX`: {str} -> unique wildcard used to identify all files
- `DATA_PATH`: {str} -> path to where all the RePORTER files are stored
- `OUTDIR`: {str} -> Path to results directory
- `OUTNAME`: {str} -> output name of the generated merged file

In [2]:
PREFIX = "RePORTER"
DATA_PATH = "../data"
OUTDIR = "results"
OUTNAME = "projects_2019-2022"

## Getting data file paths

In [3]:
# creating out directory
out_dir_path = pathlib.Path(OUTDIR)
out_dir_path.mkdir(exist_ok=True)

# setting NIH RePORTER abstract and project paths
reporter_proj_db_path = (pathlib.Path(DATA_PATH) / "RePORTER" / "projects_db").resolve(strict=True)
reporter_abs_db_path = (pathlib.Path(DATA_PATH) / "RePORTER" / "abstract_db").resolve(strict=True)

# collecting csv file paths from each data folder
proj_db_paths = [data_file for data_file in reporter_proj_db_path.glob(f"{PREFIX}*")]
abs_db_paths = [data_file for data_file in reporter_abs_db_path.glob(f"{PREFIX}*")]

## Loading data files into a pandas dataframe

The formatting of these csv files contains some lines that causes pandas tokenizer to fail
- we encode the lines, the `ignore` will remove any invalid characters that are not utf-8
- on_bad_lines is set to skip in order to prevent the C tokenization function from failing
- project_df contains all the project information from multiple files.
- this will store all dataframes and will be concatenated into a single cone

In [4]:
# loadin all Repo
project_dfs = []
for _file in proj_db_paths:
    df = pd.read_csv(
        _file,
        encoding="utf-8",
        encoding_errors="ignore",
        on_bad_lines="skip",
    )
    project_dfs.append(df)

reporter_project_df = pd.concat(project_dfs)

# printing out df metadata
rows, columns = reporter_project_df.shape
print(f"MESSAGE: Dataframe loaded {rows} rows and {columns} columns")
print("WARNING: Some entries may be omitted due to content not being utf-8 compatible")
reporter_project_df.head()

MESSAGE: Dataframe loaded 244429 rows and 46 columns


/tmp/ipykernel_25848/3489884820.py:4: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,10595864,U54,DK,6.0,N,06/13/2022,04/01/2022,07/31/2022,NaN,U54DK106829,...,106829.0,ZDK1,Special Emphasis Panel,7612.0,NaN,7.0,42060.0,31955.0,NaN,74015.0
1,10101643,R01,DA,5.0,N,02/22/2021,03/01/2021,02/28/2022,279.0,R01DA046197,...,46197.0,ZRG1,Special Emphasis Panel,NaN,NaN,4.0,451257.0,167187.0,618444.0,NaN
2,10189622,U18,FD,5.0,N,06/08/2021,06/01/2021,05/31/2022,103.0,U18FD006442,...,6442.0,ZFD1,Special Emphasis Panel,NaN,NaN,4.0,NaN,NaN,74000.0,NaN
3,10189608,U18,FD,5.0,N,06/01/2021,06/01/2021,05/31/2022,103.0,U18FD006164,...,6164.0,ZFD1,Special Emphasis Panel,NaN,NaN,5.0,NaN,NaN,52000.0,NaN
4,10076833,R01,EY,5.0,N,01/11/2021,01/01/2021,12/31/2021,867.0,R01EY015240,...,15240.0,BVS,Biology of the Visual System Study Section,NaN,NaN,16.0,335775.0,204822.0,540597.0,NaN


In [5]:
# loadin all NIH RePORTER abstract data
abstract_dfs = []
for _file in abs_db_paths:
    df = pd.read_csv(
        _file,
        encoding="utf-8",
        encoding_errors="ignore",
        on_bad_lines="skip",
    )
    abstract_dfs.append(df)

reporter_abstract_df = pd.concat(abstract_dfs)

# printing out df metadata
rows, columns = reporter_abstract_df.shape
print(f"MESSAGE: Abstract Dataframe loaded {rows} rows and {columns} columns")
print("WARNING: Some entries may be omitted due to content not being utf-8 compatible")
reporter_abstract_df.head()

MESSAGE: Abstract Dataframe loaded 238613 rows and 2 columns


,APPLICATION_ID,ABSTRACT_TEXT
0,10434631,This contract supports the advanced developmen...
1,10428336,This contract supports the advanced developmen...
2,10131817,Project Summary / Abstract The ubiquitin-prot...
3,10136452,Histone post-translational modifications (PTMs...
4,10147766,PROJECT SUMMARY/ABSTRACT Health disparities ar...


## Merging Abstact and Project dataset based on Application ID:
In this step, we'll merge the Abstract and Project datasets using the unique Application ID as the linking factor. The APPLICATION_ID serves as the distinct identifier for each project record in the RePORTER database.

This merging process will yield a single dataframe that encompasses both project and abstract data.


In [6]:
merged_reporter_df = reporter_project_df.merge(reporter_abstract_df, on="APPLICATION_ID", how="left")
merged_reporter_df.head()

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT,ABSTRACT_TEXT
0,10595864,U54,DK,6.0,N,06/13/2022,04/01/2022,07/31/2022,NaN,U54DK106829,...,ZDK1,Special Emphasis Panel,7612.0,NaN,7.0,42060.0,31955.0,NaN,74015.0,NaN
1,10101643,R01,DA,5.0,N,02/22/2021,03/01/2021,02/28/2022,279.0,R01DA046197,...,ZRG1,Special Emphasis Panel,NaN,NaN,4.0,451257.0,167187.0,618444.0,NaN,Abstract: The overdose (OD) epidemic is one of...
2,10189622,U18,FD,5.0,N,06/08/2021,06/01/2021,05/31/2022,103.0,U18FD006442,...,ZFD1,Special Emphasis Panel,NaN,NaN,4.0,NaN,NaN,74000.0,NaN,Summary: Louisiana Animal Disease Diagnostic L...
3,10189608,U18,FD,5.0,N,06/01/2021,06/01/2021,05/31/2022,103.0,U18FD006164,...,ZFD1,Special Emphasis Panel,NaN,NaN,5.0,NaN,NaN,52000.0,NaN,Project Summary The proposed work is for a coo...
4,10076833,R01,EY,5.0,N,01/11/2021,01/01/2021,12/31/2021,867.0,R01EY015240,...,BVS,Biology of the Visual System Study Section,NaN,NaN,16.0,335775.0,204822.0,540597.0,NaN,PROJECT SUMMARY Iron plays a critical role in ...


In [7]:
# save merged file
merged_save_path = pathlib.Path(f"{OUTDIR}")/f"RePORTER_merged_{OUTNAME}.csv.gz"
merged_reporter_df.to_csv(merged_save_path, index=False, compression="gzip")

# Filtering merged reporter dataset

In this steps we are only selecting columns that are helpful for us for the next step, which is the query steps. In addition, we would also like to obtain information that refers to the location, amount and project tags. For more information about that fields that are being selected please refer [here](https://api.reporter.nih.gov/)

In [8]:
# listing all columns names of the 
merged_reporter_df.columns

Index(['APPLICATION_ID', 'ACTIVITY', 'ADMINISTERING_IC', 'APPLICATION_TYPE',
       'ARRA_FUNDED', 'AWARD_NOTICE_DATE', 'BUDGET_START', 'BUDGET_END',
       'CFDA_CODE', 'CORE_PROJECT_NUM', 'ED_INST_TYPE', 'FOA_NUMBER',
       'FULL_PROJECT_NUM', 'FUNDING_ICs', 'FUNDING_MECHANISM', 'FY', 'IC_NAME',
       'NIH_SPENDING_CATS', 'ORG_CITY', 'ORG_COUNTRY', 'ORG_DEPT',
       'ORG_DISTRICT', 'ORG_DUNS', 'ORG_FIPS', 'ORG_IPF_CODE', 'ORG_NAME',
       'ORG_STATE', 'ORG_ZIPCODE', 'PHR', 'PI_IDS', 'PI_NAMEs',
       'PROGRAM_OFFICER_NAME', 'PROJECT_START', 'PROJECT_END', 'PROJECT_TERMS',
       'PROJECT_TITLE', 'SERIAL_NUMBER', 'STUDY_SECTION', 'STUDY_SECTION_NAME',
       'SUBPROJECT_ID', 'SUFFIX', 'SUPPORT_YEAR', 'DIRECT_COST_AMT',
       'INDIRECT_COST_AMT', 'TOTAL_COST', 'TOTAL_COST_SUB_PROJECT',
       'ABSTRACT_TEXT'],
      dtype='object')

In [9]:
# creating filtered_merged_df
filtered_merged_df = merged_reporter_df[["APPLICATION_ID", 
                                         "ABSTRACT_TEXT", 
                                         "ACTIVITY", 
                                         "APPLICATION_TYPE", 
                                         "AWARD_NOTICE_DATE", 
                                         "IC_NAME", 
                                         "ORG_STATE", 
                                         "ORG_ZIPCODE", 
                                         "PI_NAMEs", 
                                         "PI_IDS",
                                         "TOTAL_COST"
                                         ]]
n_entries = filtered_merged_df.shape[0]
print("started filtered:", filtered_merged_df.shape[0])

# dropping rows that do not have a total cost of funding
filtered_merged_df = filtered_merged_df.loc[~filtered_merged_df["TOTAL_COST"].isna()]
n_entries_after_drop = filtered_merged_df.shape[0] 
print("after drop:", filtered_merged_df.shape[0])

# stdout message
print(f"WARNING: total of rows dropped: {n_entries-n_entries_after_drop}")
print(f"{((n_entries-n_entries_after_drop)/n_entries)*100:.2f}% reduction!")

started filtered: 244431
after drop: 202312
17.23% reduction!


In [10]:
# display new df
filtered_merged_df.head()

,APPLICATION_ID,ABSTRACT_TEXT,ACTIVITY,APPLICATION_TYPE,AWARD_NOTICE_DATE,IC_NAME,ORG_STATE,ORG_ZIPCODE,PI_NAMEs,PI_IDS,TOTAL_COST
1,10101643,Abstract: The overdose (OD) epidemic is one of...,R01,5.0,02/22/2021,NATIONAL INSTITUTE ON DRUG ABUSE,GA,303221007,"COOPER, HANNAH LF;",7707702;,618444.0
2,10189622,Summary: Louisiana Animal Disease Diagnostic L...,U18,5.0,06/08/2021,FOOD AND DRUG ADMINISTRATION,LA,708030001,"BALASURIYA, UDENI B. R.;",10624808;,74000.0
3,10189608,Project Summary The proposed work is for a coo...,U18,5.0,06/01/2021,FOOD AND DRUG ADMINISTRATION,IA,500112025,"GAUGER, PHILLIP ;",15717248;,52000.0
4,10076833,PROJECT SUMMARY Iron plays a critical role in ...,R01,5.0,01/11/2021,NATIONAL EYE INSTITUTE,PA,191046205,"DUNAIEF, JOSHUA L;",1927397;,540597.0
5,10084900,There are a number of diseases and conditions ...,R01,5.0,03/17/2021,NATIONAL INSTITUTE OF BIOMEDICAL IMAGING AND B...,CA,900894304,"APPLEGATE, BRIAN E.;",8786272;,644204.0


In [11]:
# saving filtered merged dataset
filtered_merged_save_path = pathlib.Path(f"{OUTDIR}")/f"RePORTER_filtered_merged_{OUTNAME}.csv.gz"
merged_reporter_df.to_csv(filtered_merged_save_path, index=False, compression="gzip")
